1. Data reading & splitting

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

# reading
data = df = pd.read_csv('/content/housing-classification-iter3.csv')#housing-classification-iter3

# X and y creation
X = data
y = X.pop("Expensive")

# Feature Engineering
X.loc[:, "LotFrontage"] = X.LotFrontage

# data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

2. Categorical encoding - "MANUAL" approach (Without using Pipelines)

2.1. Replacing NaNs


We will need two different strategies to deal with missing values in numerical and categorical features.

2.1.1. Replacing NaNs in categorical features

We were imputing the mean to NaN’s on our preprocessing pipeline for numerical features. There's a problem with categorical values: they don’t have a “mean”. Here, we will replace NaNs with a string that marks them: “N_A”. It is not an elegant solution, but it will allow us to move forward.

In [5]:
# selecting non-numerical columns
X_train_cat = X_train.select_dtypes(exclude="number")

# defining the imputer to use "N_A" as replacement value
cat_imputer = SimpleImputer(strategy="constant", 
                            fill_value="N_A")

# fitting the imputer
cat_imputer.fit(X_train_cat)

# transforming the data & keeping it as a DataFrame
X_cat_imputed = pd.DataFrame(cat_imputer.transform(X_train_cat), 
                             columns=X_train_cat.columns)
X_cat_imputed.head()

,MSZoning,Condition1,Heating,Street,CentralAir,Foundation
0,RL,Norm,GasA,Pave,Y,PConc
1,RL,Norm,GasA,Pave,Y,CBlock
2,RL,PosN,GasA,Pave,Y,CBlock
3,RL,Norm,GasA,Pave,N,CBlock
4,RL,Norm,GasA,Pave,Y,Wood


2.1.2. Replacing NaNs in numerical features
This is what we already did in previous notebooks: replacing numerical NaNs with the mean of their column.

In [6]:
# Selecting numerical columns
X_train_num = X_train.select_dtypes(include="number")

# Imputing the mean
num_imputer = SimpleImputer(strategy="mean")

# Fitting
num_imputer.fit(X_train_num)

# Transforming, keeping a DataFrame
X_num_imputed = pd.DataFrame(num_imputer.transform(X_train_num), 
                             columns=X_train_num.columns)

X_num_imputed.head()

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch
0,9900.0,90.00000,1347.0,4.0,1.0,0.0,3.0,340.0,0.0
1,14585.0,69.58427,1144.0,3.0,2.0,0.0,2.0,216.0,0.0
2,12227.0,69.58427,1330.0,4.0,1.0,0.0,2.0,550.0,0.0
3,10778.0,72.00000,1768.0,4.0,0.0,0.0,0.0,0.0,0.0
4,14115.0,85.00000,796.0,1.0,0.0,0.0,2.0,40.0,0.0


In [7]:
# Concatenating all columns
X_imputed = pd.concat([X_cat_imputed, X_num_imputed], axis=1)

X_imputed.head()

,MSZoning,Condition1,Heating,Street,CentralAir,Foundation,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch
0,RL,Norm,GasA,Pave,Y,PConc,9900.0,90.00000,1347.0,4.0,1.0,0.0,3.0,340.0,0.0
1,RL,Norm,GasA,Pave,Y,CBlock,14585.0,69.58427,1144.0,3.0,2.0,0.0,2.0,216.0,0.0
2,RL,PosN,GasA,Pave,Y,CBlock,12227.0,69.58427,1330.0,4.0,1.0,0.0,2.0,550.0,0.0
3,RL,Norm,GasA,Pave,N,CBlock,10778.0,72.00000,1768.0,4.0,0.0,0.0,0.0,0.0,0.0
4,RL,Norm,GasA,Pave,Y,Wood,14115.0,85.00000,796.0,1.0,0.0,0.0,2.0,40.0,0.0


2.2. One Hot encoding

As you have learnt in the Platform lesson, One Hot encoding means creating a new binary column for each category in every categorical column. Fortunately, a Scikit-Learn transformer takes care of everything.

2.3.1. Fitting the OneHotEncoder
As with any transformer, we have to:

1. Import it
2. Initialize it
3. Fit it to the data
4. Use it to transform the data

In [8]:
# import
from sklearn.preprocessing import OneHotEncoder

# initialize
my_onehot = OneHotEncoder(drop="first")

# fit
my_onehot.fit(X_cat_imputed)

# transform
X_cat_imputed_onehot = my_onehot.transform(X_cat_imputed)

The result is a "sparse matrix": an object that Scikit-Learn creates when a matrix contains mostly zeros:

In [9]:
X_cat_imputed_onehot

<1168x24 sparse matrix of type '<class 'numpy.float64'>'
	with 6762 stored elements in Compressed Sparse Row format>

2.3.2. Converting the sparse matrix into a DataFrame

To see what exactly is inside of this sparse matrix we can convert it to a pandas DataFrame:

In [10]:
df = pd.DataFrame.sparse.from_spmatrix(X_cat_imputed_onehot)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


We can see that all the columns contain either 0's or 1's. This is exactly how "one-hot" encoded columns (also called "dummy columns") look like: binary categories.

Now, for exploration and learning purposes, we will rename the columns in this dataframe so that we know the origin of each binary column (the category and original column they come from).

2.3.3. Retrieving the column names for the "one-hot" columns

The fitted transformer contains this information, and the method get_feature_names_out allows us to recover the names of the columns.

Note: If you're running this code as a local Jupyter notebook and you don't have the last version of Scikit-Learn, you might have to adapt the code. Check the documentation for the Scikit-Learn version you have installed.

In [11]:
colnames = my_onehot.get_feature_names_out(X_cat_imputed.columns)
df.columns = colnames
df.head()

,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,...,Heating_Grav,Heating_OthW,Heating_Wall,Street_Pave,CentralAir_Y,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


Each column has the name of the original column, an underscore and the name of the category:

A column such as "Sex", with only two categories, "male" and "female", has become a single column, "Sex_male", where 1 stands for "male" and 0 for "female".

A column such as "Cabin", with many categories ("A", "B", "C", "D", "E", "F", "G", "N_A", "T") has become as many columns as categories were (minus one), making the dataframe very wide and sparse.

2.3.4. Concatenating "one-hot" columns with numerical columns:

Now that the categorical columns are numerical, we can join them back with the originally numerical columns and assemble the dataset that will be ready for modelling:

In [12]:
X_imputed = pd.concat([df, X_num_imputed], axis=1)

X_imputed.head(3)

,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,...,Foundation_Wood,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,9900.0,90.00000,1347.0,4.0,1.0,0.0,3.0,340.0,0.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,14585.0,69.58427,1144.0,3.0,2.0,0.0,2.0,216.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,12227.0,69.58427,1330.0,4.0,1.0,0.0,2.0,550.0,0.0


3. Categorical encoding - "Automated" approach (Using Pipelines)

In the manual approach, to encode the categorical columns numericall, we have:

1. Selected the categorical columns.
2. Fitted a OneHotEncoder to them.
3. Transformed the categorical columns with the encoder.
4. Converted the sparse matrix into a dataframe.
5. Recovered the names of the columns.
6. Concatenated the one-hot columns with the numerical columns.

All these steps can be synthetised by using Scikit-Learn Pipelines and specifically something called ColumnTransformer, which allows us to apply different transformations to two or more groups of columns: in our case, categorical and numerical columns.

This process is also called creating "branches" in the pipeline. One branch for the categorical columns and another for the numerical columns. Each branch will contain as many transformers as we want. Then, the branches will meet again, and the transformed columns will be automatically concatenated. Let's see the process in action:

3.1. Creating the "numeric pipe" and the "categoric pipe"

In [13]:
# select categorical and numerical column names
X_cat_columns = X.select_dtypes(exclude="number").copy().columns
X_num_columns = X.select_dtypes(include="number").copy().columns

# create numerical pipeline, only with the SimpleImputer(strategy="mean")
numeric_pipe = make_pipeline(
    SimpleImputer(strategy="mean"))
 
 # create categorical pipeline, with the SimpleImputer(fill_value="N_A") and the OneHotEncoder
categoric_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="N_A"),
    OneHotEncoder()
)

3.2. Using ColumnTransformer a pipeline with 2 branches (the preprocessor)

We simply tell the pipeline the following:

One branch, called "num_pipe", will apply the steps in the numeric_pipe to the columns named in X_num_columns
The second branch, called "cat_pipe", will apply the steps in the categoric_pipe to the columns named in X_cat_columns

In [14]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("num_pipe", numeric_pipe, X_num_columns),
        ("cat_pipe", categoric_pipe, X_cat_columns),
    ]
)

3.3. Creating the full_pipeline (preprocessor + Decision Tree)

Pipelines are modular. The preprocessor we created above with the ColumnTransformer can become now a step in a new pipeline, that we'll call full_piepline and will include, as a last step, a Decision Tree model:

In [15]:
full_pipeline = make_pipeline(preprocessor, 
                              DecisionTreeClassifier())

We can then fit this full_pipeline to the data:

Note: notice that we did not fit the preprocessor before —we only fit the pipeline once it has been full assembled.

In [16]:
full_pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_pipe',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer())]),
                                                  Index(['LotArea', 'LotFrontage', 'TotalBsmtSF', 'BedroomAbvGr', 'Fireplaces',
       'PoolArea', 'GarageCars', 'WoodDeckSF', 'ScreenPorch'],
      dtype='object')),
                                                 ('cat_pipe',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='N_A',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(

This full pipeline can make predictions, as any othet pipeline that ends with a model:

In [17]:
full_pipeline.predict(X_train)

array([1, 0, 1, ..., 1, 0, 0])